<a href="https://colab.research.google.com/github/btripura/VQA_MODEL/blob/main/VQA_TRAINING_VILBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wget
!pip install datasets==1.17.0 nltk==3.5 pandas==1.3.5 Pillow scikit-learn==0.23.2 torch==1.12.0 transformers==4.15.0 dvc==2.9.3

In [ ]:
!pip install --pre allennlp-models

In [ ]:
import cv2
import numpy as np
import math
from google.colab import files
from IPython.display import Image
from google.colab.patches import cv2_imshow

In [ ]:
uploaded = files.upload()

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
imdb="gdrive/MyDrive"
from allennlp_models.pretrained import load_predictor
predictor = load_predictor("vqa-vilbert")

In [ ]:
import os
from typing import Dict, List, Optional, Tuple
import numpy as np
from PIL import Image

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.image import load_img, img_to_array,array_to_img

In [ ]:
from collections import deque
import cv2, numpy as np
import time
import math as mth
from PIL import Image
import random
import argparse
from hmac import trans_36
from shutil import move
from socket import CAN_BCM_RX_CHANGED
from cv2 import detail_AffineBasedEstimator
from keras.models import Sequential
from keras import initializers
from keras.initializers import normal, identity
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.recurrent import LSTM
from tensorflow.keras.optimizers import RMSprop, SGD, Adam
from tensorflow.keras.preprocessing.image import load_img, img_to_array


from IPython.display import Image,display
from google.colab.patches import cv2_imshow
from tensorflow.keras.preprocessing.image import array_to_img
import PIL.Image
import io
from transformers import VisualBertForQuestionAnswering, BertTokenizerFast
import torch

In [ ]:
train_q=[]
with open('train_questions.txt','r') as fp:
  for line in fp:
    x=line[:-1]
    train_q.append(x)

train_a=[]
with open('train_answers.txt','r') as fp:
  for line in fp:
    x=line[:-1]
    train_a.append(x)

train_id=[]
with open('train_image_ids.txt','r') as fp:
  for line in fp:
    x=line[:-1]
    train_id.append(x)

In [ ]:
def load_and_process_image(image_path):
       im = PIL.Image.open(image_path)
       im=im.resize((224,224))
       im=np.array(im)
       return im
def read_images():
        ims=[]
        for i in range(70000):
          image_name=train_id[i].strip()
          ims.append(load_and_process_image(os.path.join(imdb, "train_images", image_name)))
        return ims

train_ims=read_images()
print(len(train_ims))
img_shape=train_ims[0].shape
print(img_shape)



In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_q)

# We add one because the Keras Tokenizer reserves index 0 and never uses it.
vocab_size = len(tokenizer.word_index) + 1

print('\n--- Converting questions to bags of words...')
train_X_seqs = tokenizer.texts_to_matrix(train_q)


--- Converting questions to bags of words...


In [ ]:
num_actions=5
observing_episodes=20
# How many steps can run the agent until finding one object
number_of_steps = 10
# Boolean to indicate if you want to use the two databases, or just one
episodes = 70000
gamma = 0.90
epsilon_current_value = 1
epsilon_initial_value=1
epsilon_final_value=0.0001
batch_size = 100
replay_memory = 5000
transitions=deque()

number_of_actions = 5
delta=10


#Image Dimensions
H=224
W=224

X = 0
Y = 0


st_x=0
e_x=W
st_y=0
e_y=H

xval=0
yval=0

In [ ]:
from PIL import Image
def transform(image,x_val,y_val):
    print('Transforming Image')
    global X
    global Y
    X=x_val
    Y=y_val
    M=np.float32([[1,0,X],[0,1,Y]])
    output=cv2.warpAffine(image,M,(224,224))
    fdf=[X,Y]
    return cropped(output)

def cropped(image):
        start_x=st_x
        start_y=st_y
        end_x=e_x
        end_y=e_y
        #print('cropping\n')
        values=[X,Y]
        #print(values)
        if(X<0):
           end_x=W+X
        if(X>0):
           start_x=X
        if(Y<0):
           end_y=H+Y
        if(Y>0):
           start_y=Y
        crop=image[start_y:end_y,start_x:end_x]
        resized=cv2.resize(crop,(W,H),interpolation=cv2.INTER_NEAREST)
        #print('after crop')
        #cv2_imshow(resized)
        return resized

In [ ]:
def predict(image,ques):
    print('Predicting Answer')
    URL='temp.jpg'
    preds = predictor.predict(URL, ques)
    best_prob, best_answer = max(zip(preds["probs"], preds["tokens"]), key=lambda x: x[0])
    print(f"p({best_answer}) = {best_prob:.2%}")
    print("answer:--",best_answer)
    print("Conf---",best_prob)
    return (best_answer,(best_prob/100))

In [ ]:
def get_reward(image,ques,ans,prev_conf):
    print('Calculating Reward')
    image=array_to_img(image)
    #print(image.size)
    image.save('temp.jpg')
    #print('saved')
    predict_ans,new_conf=predict(image,ques)
    #print('Question: ',ques)
    #print('Answer: ', predict_ans)
    if(ans==predict_ans):
      return (new_conf-prev_conf,new_conf)
    else:
       return (-1,0)

In [ ]:
def step(state,ques,ans,conf,action):
    #print('action taking---',action)
    print('Performing action')
    global xval
    global yval
    xval=X
    yval=Y
    if action==0:
        xval+=delta
    elif action==1:
        xval-=delta
    elif action==2:
        yval+=delta
    elif action==3:
        yval-=delta
    elif action==4:
      pass
    #print('IN step:')
    vad=[xval,yval]
    #print(vad)
    new_image=transform(state[0],xval,yval)

    reward,new_conf=get_reward(new_image,ques,ans,conf)
    new_image=img_to_array(new_image)
    return (new_image,reward,new_conf)

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten, Multiply,Embedding
from tensorflow.keras.optimizers import Adam
def create_model(im_shape, vocab_size, num_actions):
  print('creating model---\n')
  im_input = Input(shape=im_shape)
  x1 = Conv2D(8, 3, padding='same')(im_input)
  x1 = MaxPooling2D()(x1)
  x1 = Conv2D(16, 3, padding='same')(x1)
  x1 = MaxPooling2D()(x1)
  x1 = Flatten()(x1)
  x1 = Dense(1000, activation='relu')(x1)
  # The question network
  q_input = Input(shape=(vocab_size,))
  embedded_question = Embedding(input_dim=10000, output_dim=256, input_length=100)(q_input)
  x2 = LSTM(256)(embedded_question)
  x2 = Dense(1000, activation='relu')(x2)
  # Merge -> output
  out = Multiply()([x1, x2])
  out = Dense(1000, activation='relu')(out)
  out = Dense(num_actions, activation='softmax')(out)
  model = Model(inputs=[im_input, q_input], outputs=out)
  model.compile(Adam(lr=5e-4), loss='categorical_crossentropy', metrics=['accuracy'])
  return model

In [ ]:
from tensorflow.keras.models import load_model
model=create_model(img_shape,vocab_size,num_actions)
#model=load_model('gdrive/MyDrive/model_trained.h5')

In [ ]:
def remember(state,action,next_state,reward):      #This is the function to store our experiences
    print('Remembering state')
    global transitions
    transitions.append([state,action,next_state,reward])
    #print(len(transitions))
    if len(transitions) > replay_memory:
      transitions.popleft()

In [ ]:
def get_action(state):
    print('Getting action')
    x=random.random()
    if x < epsilon_current_value:
        #print('random--\n')
        action=random.choice([0,1,2,3,4])
        #print(action)

    else:
        q_values=model.predict([np.array([state[0]]),np.array([state[1]])]) #Exploitation
        max_Q = np.argmax(q_values[0])
        action = max_Q
        #print('not random',action)

    return action

In [ ]:
#from tensorflow.keras.callbacks import ModelCheckpoint
#checkpoint = ModelCheckpoint('model.h5', save_best_only=True)
def update_policy():
    print('Updating policy---\n')
    global transitions
    transitions=random.sample(transitions,batch_size)
    #print(len(transitions))
    train_x_ims=[]
    train_x_ques=[]
    targets = np.zeros((batch_size,5))
    train_x_ims=np.array([transitions[id][0][0] for id in range(batch_size)])
    train_x_ques=np.array([transitions[id][0][1] for id in range(batch_size)])
    targets=model.predict([train_x_ims,train_x_ques])
    train_x_next_ims=np.array([transitions[id][2][0] for id in range(batch_size)])
    train_x_next_ques=np.array([transitions[id][2][1] for id in range(batch_size)])
    Q_sa=model.predict([train_x_next_ims,train_x_next_ques])
    for i in range(0,batch_size):
      action=transitions[i][1] #This is action
      reward=transitions[i][3] #reward at state due to action
      targets[i, action] = reward + np.asarray(gamma) * np.max(Q_sa[i])
    model.fit([train_x_ims,train_x_ques], targets,epochs=50,) #Training the model

In [ ]:
def reset():
  global X
  global Y
  global st_x
  global st_x
  global e_x
  global st_y
  global e_y
  global xval
  global yval
  xval=0
  yval=0
  X=0
  Y=0
  st_x=0
  e_x=224
  st_y=0
  e_y=224

In [ ]:
for episode in range(episodes):
    state=[train_ims[episode],train_X_seqs[episode]]
    episode_reward=0
    conf=0
    print('Episode: ',episode)
    reset()
    for i in range(number_of_steps):
        print("Step: ",i)
        action=get_action(state)
        new_image,reward,new_conf=step(state,train_q[episode],train_a[episode],conf,action)
        next_state=[new_image,train_X_seqs[episode]]
        conf=new_conf
        print("Episode Going On."+"\n"+"Action taken:"+'\t',action)
        remember(state,action,next_state,reward)
        state=next_state
        episode_reward+=reward
    print("Episode_reward:{}".format(episode_reward))
    print('Current Epsilon Value:',epsilon_current_value)
    epsilon_current_value=epsilon_current_value-(epsilon_initial_value-epsilon_final_value)/1000
    print("Episode Ended")
    if (episode+1)%observing_episodes==0 and episode!=0:
        update_policy()
        model.save('model_2_{}.h5'.format(episode))
        #files.download('model_2_{}.h5'.format(episode))